In [54]:
# import getpass
import os
import json
import pandas as pd

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
KEY=os.getenv("GROQ_API_KEY")


In [89]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# !pip install -qU langchain-groq
from langchain_groq import ChatGroq


model = ChatGroq(groq_api_key= KEY, model="llama3-8b-8192")
parser = StrOutputParser()
model_parser = model | parser

In [56]:
RESPONSE_JSON = {
    "1": {
        "STATSQA": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "STATSQA": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "STATSQA": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [79]:
TEMPLATE="""
Text:{text}
You are an expert STATSQA maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions related to {area} for students in {grade}. 
Make sure the questions are not repeated and format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json} \

"""

In [80]:
TEMPLATE2="""
You are an expert of English grammar. \
You are given a STATSQA quiz: {quiz} of multiple choice questions related to {area} in statistics.\
You need to evaluate the complexity of the quiz. Use at most 50 words for complexity analysis. \
If the quiz is too easy or too difficult for students in {grade}, 
update the quiz questions to make it more suitable for the students in {grade}.

Check from an expert English Writer of the above quiz:
"""

In [90]:
quiz_generation_prompt = ChatPromptTemplate.from_template(
    template=TEMPLATE
)

In [91]:
quiz_evaluation_prompt = ChatPromptTemplate.from_template(TEMPLATE2)

In [93]:
quiz_generator = (
    {"text": RunnablePassthrough("text"),
    "number": itemgetter("number"),
    "area": itemgetter("area"),
    "grade": itemgetter("grade"),
    "response_json": json.dumps(RESPONSE_JSON, indent=4)}
    | quiz_generation_prompt
    | {"quiz": model_parser}
)

ValidationError: 2 validation errors for RunnablePassthrough
func
  text is not callable (type=type_error.callable; value=text)
func
  text is not callable (type=type_error.callable; value=text)

: 

In [ ]:
quiz_evaluator = quiz_generator | quiz_evaluation_prompt | model_parser

In [ ]:
quiz_evaluator.invoke(
    text="You are an expert STATSQA maker. Given the above text, it is your job to create a quiz of 3 multiple choice questions related to statistics for students in grade 10. Make sure the questions are not repeated and format your response like RESPONSE_JSON below and use it as a guide. Ensure to make 3 MCQs",
    number="3",
    area="probability",
    grade="high school",
    response_json=json.dumps(RESPONSE_JSON, indent=4)
)

In [70]:
prompt1 = PromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = PromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = PromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = PromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [73]:
question_generator.invoke("warm")

StringPromptValue(text='What is the color of Tangerine and the flag of Nauru?')

In [74]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color of a pomegranate is typically a deep red or reddish-purple color.\n\nThe flag of Italy is a tricolor, featuring three vertical stripes of green, white, and red, in that order.', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 25, 'total_tokens': 70, 'completion_time': 0.035103997, 'prompt_time': 0.004526317, 'queue_time': None, 'total_time': 0.039630314}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-a9782277-6e03-47ba-84c4-a023d55842b2-0')

In [36]:
# Example usage of the created prompt_template object
# Assuming there's a method to fill in the template with actual values
first_prompt = quiz_generation_prompt.invoke(
    {"text": "The concept of probability in statistics is a way to quantify uncertainty.",
     "number": 3,
     "area": "probability",
     "grade": "high school",
     "response_json": json.dumps(RESPONSE_JSON, indent=4)}
)

print(first_prompt)

text='\nText:The concept of probability in statistics is a way to quantify uncertainty.\nYou are an expert STATSQA maker. Given the above text, it is your job to create a quiz  of 3 multiple choice questions related to probability for students in high school. \nMake sure the questions are not repeated and format your response like RESPONSE_JSON below and use it as a guide. Ensure to make 3 MCQs\n### RESPONSE_JSON\n{\n    "1": {\n        "STATSQA": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "2": {\n        "STATSQA": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "3": {\n        "STATSQA": "multipl

In [37]:
quiz_generation_chain = quiz_generation_prompt | model | parser

In [40]:
first_chain_result = quiz_generation_chain.invoke(
{"text": "point estimation, in statistics, the process of finding an approximate value of some parameter—such as the mean (average)—of a population from random samples of the population.",
     "number": 3,
     "area": "estimation",
     "grade": "high school",
     "response_json": json.dumps(RESPONSE_JSON, indent=4)}

)
print(first_chain_result)

Here are 3 multiple choice questions related to estimation:

{
    "1": {
        "STATSQA": "What is point estimation in statistics?",
        "options": {
            "a": "The process of finding the exact value of a population parameter",
            "b": "The process of finding an approximate value of a population parameter",
            "c": "The process of finding the mean of a sample",
            "d": "The process of finding the median of a sample"
        },
        "correct": "b"
    },
    "2": {
        "STATSQA": "What is the goal of point estimation?",
        "options": {
            "a": "To find the exact value of a population parameter",
            "b": "To find an approximate value of a population parameter",
            "c": "To find the standard deviation of a sample",
            "d": "To find the correlation coefficient of a sample"
        },
        "correct": "b"
    },
    "3": {
        "STATSQA": "What type of sample is typically used for point estimation?

## 2. Second Chain - Quiz Evaluation and Update

In [43]:
second_prompt = quiz_evaluation_prompt.invoke(
    {"area": "probability",
     "grade": "high school",
     "quiz": first_chain_result}
)

print(second_prompt)

text='\nYou are an expert of English grammar. You are given a STATSQA quiz of multiple choice questions related to probability in statistics.You need to evaluate the complexity of the quiz. Use at most 50 words for complexity analysis. If the quiz is too easy or too difficult for students in high school, \nupdate the quiz questions to make it more suitable for the students in high school.\nQuiz_MCQs:\nHere are 3 multiple choice questions related to estimation:\n\n{\n    "1": {\n        "STATSQA": "What is point estimation in statistics?",\n        "options": {\n            "a": "The process of finding the exact value of a population parameter",\n            "b": "The process of finding an approximate value of a population parameter",\n            "c": "The process of finding the mean of a sample",\n            "d": "The process of finding the median of a sample"\n        },\n        "correct": "b"\n    },\n    "2": {\n        "STATSQA": "What is the goal of point estimation?",\n       

In [46]:
quiz_evaluation_chain = quiz_evaluation_prompt | model | parser

In [47]:
second_chain_result = quiz_evaluation_chain.invoke(
    {"area": "probability",
     "grade": "high school",
     "quiz": quiz_generation_chain}
)

print(second_chain_result)

Based on the quiz questions, I would rate the complexity level as moderately challenging for high school students. The questions appear to require a good understanding of probability concepts, such as conditional probability, independence, and Bayes' theorem. However, the language used is straightforward, and the options provided are clear and concise.

To make the quiz more suitable for high school students, I would consider adding more contextual clues to help them understand the problems better. For example, adding simple diagrams or charts to illustrate the probability concepts could be helpful.

Here is a revised complexity analysis:

* Complexity level: 6/10 (moderately challenging)
* Recommended adjustments: Add more contextual clues, such as diagrams or charts, to help students understand the probability concepts.

Please let me know if you would like me to update the quiz questions based on this analysis.


In [21]:
print(quiz_generation_chain)

Here are the 3 multiple choice questions related to estimation:

{
    "1": {
        "STATSQA": "What is the process of finding an approximate value of some parameter of a population?",
        "options": {
            "a": "Sampling",
            "b": "Point Estimation",
            "c": "Hypothesis Testing",
            "d": "Regression Analysis"
        },
        "correct": "b"
    },
    "2": {
        "STATSQA": "What is the typical parameter that is estimated in point estimation?",
        "options": {
            "a": "Standard Deviation",
            "b": "Mean (Average)",
            "c": "Median",
            "d": "Mode"
        },
        "correct": "b"
    },
    "3": {
        "STATSQA": "What is the basis of point estimation?",
        "options": {
            "a": "Random Sampling",
            "b": "Systematic Sampling",
            "c": "Stratified Sampling",
            "d": "Convenience Sampling"
        },
        "correct": "a"
    }
}


In [ ]:

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=quiz_generation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "area":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )